<a href="https://colab.research.google.com/github/DeepakSaiMadishetty/DeepakSaiMadishetty.github.io/blob/master/object_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import cv2
import tensorflow as tf
from keras.applications.vgg16 import VGG16
from keras.layers import Conv2D
from keras.layers import MaxPooling2D

Using TensorFlow backend.


In [0]:
# since we are about to perform transfer learning , we do not require the topmost layer.
# also , as per out image size, which is 460*640 we adjust the model input size
# while top layer is included , if input size is altered , an error is shown

inp=[460,640]
intialModel=VGG16(weights='imagenet',include_top=False,input_shape=inp+[3])

In [0]:
# we can observe that the top layer(flatten) is not included
# also that the model is adjusted to an image input 460*640
for layer in intialModel.layers:
    layer.trainable=False
intialModel.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 460, 640, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 460, 640, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 460, 640, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 230, 320, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 230, 320, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 230, 320, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 115, 160, 128)     0     

In [0]:
# here we need to add inceptionn modules to the model
# an inception module is a set of parallel layers which give their prediction to the final layer directly
# In this project, we include 4 inception modules with their filter sizes being 1X1,3X3,5X5 and 7X7.

def add_conv(no_of_layers,one_or_three,inlayer,filters):
    a=one_or_three
    conv=inlayer
    for _ in range(no_of_layers):
        conv=Conv2D(filters,(one_or_three,one_or_three),padding="same",activation="relu")(conv)
    return conv
    

In [0]:
# instead of adding 5X5 and 7X7 ,here we are adding two and three 3X3 conv2D layers respectively
from keras.layers import concatenate
def add_inception(inlayer):
    
    # one 1X1 convolution layer
    conv1=add_conv(1,1,inlayer,1024)
    
    # one 3X3 convolution layer
    conv3=add_conv(1,3,inlayer,1024)
    
    # two 3X3 convolution layers
    conv5=add_conv(2,3,inlayer,1024)
    
    # three 3X3 convolution layers
    conv7=add_conv(3,3,inlayer,1024)
    
    # concatenate all four inceptions.
    inception_layer=concatenate([conv1,conv3,conv5,conv7],axis=-1)
    return inception_layer

In [0]:
from keras.models import Model
inception_layer=add_inception(intialModel.output)
#print(intialModel.output)
model=Model(input=intialModel.input,output=inception_layer)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 460, 640, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 460, 640, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 460, 640, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 230, 320, 64) 0           block1_conv2[0][0]               
____________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  after removing the cwd from sys.path.


In [0]:
# now, we need to import the dataset to perform training and testing operations on the model

# here, wea re using KITTI and COCO datasets to train the model
import tensorflow_datasets as tfds

In [0]:
ds=tfds.load('kitti')

In [0]:
!git clone https://github.com/tensorflow/datasets.git
%cd datasets
# !git checkout vijayphoenix-patch-1 # PR #1427 https://github.com/tensorflow/datasets/pull/1427
!pip install -e .

In [0]:
!python tensorflow_datasets/scripts/download_and_prepare.py --datasets=kitti

In [0]:
import tensorflow as tf
import tensorflow_datasets as tfds
import json
# ======
# Load the dataset with TensorFlow Datasets.
dataset= tfds.load('kitti')

# Create a training set, a validation set and a test set.
training, testing, validation = dataset['train'], dataset['test'], dataset['validation']

In [0]:
print(model.output)

Tensor("concatenate_1/concat:0", shape=(None, 14, 20, 4096), dtype=float32)


In [0]:
from keras.layers import Flatten
from keras.layers import Dense

In [0]:
flatten_layer=Flatten()(model.output)

In [0]:
print(Flatten)

<class 'keras.layers.core.Flatten'>


In [0]:
prediction_layer=Dense(1000,activation='softmax')(flatten_layer)

In [0]:
print(prediction_layer)

In [0]:
model=Model(input=model.input,output=prediction_layer)

In [0]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 460, 640, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 460, 640, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 460, 640, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 230, 320, 64) 0           block1_conv2[0][0]               
____________________________________________________________________________________________